In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, KFold, TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import joblib


USE_STANDARDIZATION = 0       #####  1 做标准化，0 不做标准化
CV_TYPE = 2       ##### 交叉验证 1 KFold，2 时间顺序
FOLD_NUM = 5      ##### 折数

dataset_s = pd.read_csv('./test/RB99_1m_Turnover_31000_12120_1213.csv_tz80_Train_10877_ab.csv')    ####### 训练集文件
dataset = dataset_s

num_xunlian = len(dataset_s)
data_1_size = 1213    ###### 测试数据行数  ###############
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000       ###### 总资金 ###################

### 模型训练指标保存
res1 = []
res2 = []
res3 = []
res4 = []
res5 = []
res6 = []
res7 = []
resP_A = []
resP_B = []
resR_A = []
resR_B = []
resF_A = []
resF_B = []


for j in range(1, 21):    ########## 从n维训练到多少维
    num = j
    
    X = dataset.drop(['A0', 'B0'], axis=1)
    y = dataset[['A0', 'B0']]
    
    ### 是否标准化
    if USE_STANDARDIZATION == 1:
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X)
    else:
        scaler = None
        X_processed = X.copy()
    
    ### PCA降维
    pca = PCA(n_components=num, random_state=369)
    X_pca = pca.fit_transform(X_processed)
    
    ### XGBoost模型多标签回归
    base_model = xgb.XGBRegressor(random_state=369)
    model = MultiOutputRegressor(base_model)
    
    ### 定义参数网格进行调优
    param_grid = {
        'estimator__n_estimators': [100, 200],
        'estimator__learning_rate': [0.01, 0.1],
        'estimator__max_depth': [3, 5]
    }
    
    ### 交叉验证
    if CV_TYPE == 1:
        cv = KFold(n_splits=FOLD_NUM, shuffle=True, random_state=369)
    else:
        cv = TimeSeriesSplit(n_splits=FOLD_NUM)
    
    ### 网格搜索
    grid_search = GridSearchCV(
        model, 
        param_grid, 
        cv=cv, 
        scoring='neg_root_mean_squared_error', 
        n_jobs=-1
    )
    grid_search.fit(X_pca, y)
    best_model = grid_search.best_estimator_
    
    ### 交叉验证
    if CV_TYPE == 1:
        ### KFold可用cross_val_predict
        cv_preds = cross_val_predict(best_model, X_pca, y, cv=cv)
    else:
        ####  时间顺序
        cv_preds = np.zeros_like(y.values)  
        seen_indices = set()  
        
        for train_idx, test_idx in cv.split(X_pca):
            new_test_idx = [idx for idx in test_idx if idx not in seen_indices]
            if not new_test_idx:
                continue
            
            best_model.fit(X_pca[train_idx], y.iloc[train_idx])
            preds = best_model.predict(X_pca[new_test_idx])
            
            ####  保存结果
            cv_preds[new_test_idx] = preds
            seen_indices.update(new_test_idx)
    
    ##### 计算A0和B0的指标
    cv_mse_A = mean_squared_error(y['A0'], cv_preds[:, 0])
    cv_rmse_A = np.sqrt(cv_mse_A)
    cv_r2_A = r2_score(y['A0'], cv_preds[:, 0])
    
    cv_mse_B = mean_squared_error(y['B0'], cv_preds[:, 1])
    cv_rmse_B = np.sqrt(cv_mse_B)
    cv_r2_B = r2_score(y['B0'], cv_preds[:, 1])
    
    ### 保存指标
    metrics_df_A = pd.DataFrame({
        'Metric': ['MSE', 'RMSE', 'R2'],
        'Mean': [cv_mse_A, cv_rmse_A, cv_r2_A],
        'Std': [0, 0, 0]
    })
    metrics_df_B = pd.DataFrame({
        'Metric': ['MSE', 'RMSE', 'R2'],
        'Mean': [cv_mse_B, cv_rmse_B, cv_r2_B],
        'Std': [0, 0, 0]
    })
    metrics_df_A.to_csv(f'./temp/{j}_A_r.csv', index=False)
    metrics_df_B.to_csv(f'./temp/{j}_B_r.csv', index=False)
    
    #### 保存模型
    pipeline = {
        'use_standardization': USE_STANDARDIZATION,
        'scaler': scaler,
        'pca': pca,
        'model': best_model,
        'cv_type': CV_TYPE,
        'fold_num': FOLD_NUM
    }
    joblib.dump(pipeline, f'./temp/{num}_x.pkl')
    
    ### 加载测试数据
    data = pd.read_csv('./test/RB99_1m_Turnover_31000_12120_1213.csv_tz80_Test_1213_PCA.csv')     ###### 测试集数据
    pipeline = joblib.load(f'./temp/{num}_x.pkl')
    use_std = pipeline['use_standardization']
    scaler = pipeline['scaler']
    pca = pipeline['pca']
    model = pipeline['model']
    
    X_test = data.drop(['A0', 'B0'], axis=1) if 'A0' in data.columns else data.drop(['B0'], axis=1, errors='ignore')
    if use_std == 1:
        X_test_processed = scaler.transform(X_test)
    else:
        X_test_processed = X_test.copy()
    
    X_test_pca = pca.transform(X_test_processed)
    preds = model.predict(X_test_pca)
    
    ### 归一化确保 A0+B0=1
    preds_sum = preds.sum(axis=1, keepdims=True)
    preds = preds / preds_sum
    A0_preds = preds[:, 0]
    B0_preds = preds[:, 1]
    
    ### 保存推理结果
    data['A0_pred'] = A0_preds
    data['B0_pred'] = B0_preds
    A0_n_preds = data['A0_pred'][num_xunlian : num_xunlian + data_1_size].reset_index(drop=True)
    B0_n_preds = data['B0_pred'][num_xunlian : num_xunlian + data_1_size].reset_index(drop=True)
    
    with open(f'./temp/{num}_A.txt', 'a') as f:
        for val in A0_n_preds:
            f.write(f'{val}\n')
    with open(f'./temp/{num}_B.txt', 'a') as f:
        for val in B0_n_preds:
            f.write(f'{val}\n')
    
    #### 信号处理
    file_name = './temp/Show.csv'
    df = pd.read_csv(file_name)
    path_A = f'./temp/{j}_A.txt'
    df_A = pd.read_csv(path_A, header=None, names=['A0_pred'])
    path_B = f'./temp/{j}_B.txt'
    df_B = pd.read_csv(path_B, header=None, names=['B0_pred'])
    df['A0_pred'] = df_A['A0_pred']
    df['B0_pred'] = df_B['B0_pred']
    df['low'] = np.where(df['A0_pred'] > df['B0_pred'], 1, 0)
    df.to_csv(f'./temp/{j}_x.csv', index=False)
    
    ### 可加入过滤信号逻辑(可选)  ###########################################
    
    
    
    
    file_name = f'./temp/{j}_x.csv'
    data_1_new = pd.read_csv(file_name)
    
    if buy == 0:
        for i in range(0, data_1_size):
            if data_1_new.loc[i, 'low'] == 1:
                data_1_new.loc[i, 'volume'] = data_1_new.loc[i, 'volume'] * -1
    else:
        for i in range(0, data_1_size):
            if data_1_new.loc[i, 'low'] == 0:
                data_1_new.loc[i, 'volume'] = data_1_new.loc[i, 'volume'] * -1
            if data_1_new.loc[i, 'low'] == 2:    ###### 过滤信号，2为不持仓
                data_1_new.loc[i, 'volume'] = 0
    
    data_1_new['high'] = data_1_new['volume'].cumsum()
    data_1_new['open'] = rrr * data_1_new['high'] / m
    
    ######################################################################################################

    ### 胜率
    wp_win = data_1_new['volume'] > 0
    wp_lost = data_1_new['volume'] < 0
    wp_nothing = data_1_new['volume'] == 0

    wp_win_a = wp_win.sum()            
    wp_lost_a = wp_lost.sum()
    wp_nothing_a = wp_nothing.sum()

    ### 盈亏比
    rrr_win = data_1_new.loc[wp_win, 'volume'].sum()
    rrr_lost = data_1_new.loc[wp_lost, 'volume'].sum()

    ##############################################################################################
    ###### 计算回撤数据
    data_1_new['cum_max_open'] = data_1_new['open'].cummax()  
    data_1_new['down'] = data_1_new['open'] - data_1_new['cum_max_open']  
    data_1_new['down'] = data_1_new['down'].clip(upper=0)  

    ##############################################################################################
    ###### 计算回撤面积
    downarea = data_1_new['down'].sum()

    ##############################################################################################
    ### 二级模型预留
    

    data_1_new['re'] = data_1_new['close'].pct_change() * 100
    data_1_new['real'] = (data_1_new['close'] >= data_1_new['close'].shift(1)).astype(int)
    data_1_new.loc[0, 'real'] = 0 

    if buy == 0:
        data_1_new['real_lab'] = np.where(
            data_1_new['low'] != data_1_new['real'], 
            'G', 'N'
        )
    else:
        data_1_new['real_lab'] = np.where(
            data_1_new['low'] == data_1_new['real'], 
            'G', 'N'
        )

    data_1_new.loc[0, 'real_lab'] = 'G' 

    file_name = './temp/Show.csv'
    df = pd.read_csv(file_name)        
    data_1_new['show'] = df['low']

    data_1_new['show_lab'] = np.where(
        data_1_new['low'] == data_1_new['show'], 
        'G', 'N'
    )

    data_1_new.loc[0, 'show_lab'] = 'G'  

    ##############################################################################################


    if sell == 0:
        data_1_new['re_real'] = np.where(
            data_1_new['low'] == 0, 
            -data_1_new['re'], 
            data_1_new['re']
        )
    else:
        data_1_new['re_real'] = np.where(
            data_1_new['low'] == 1, 
            -data_1_new['re'], 
            data_1_new['re']
        )

    data_1_new.loc[0, 're_real'] = 0  

    ##############################################################################################

    ###### 计算夏普比率和索提诺比率
    re_real = data_1_new['re_real'][1:]  
    mean_re = re_real.mean()
    std_re = re_real.std()
    sharpe = round(mean_re / std_re * 100 if std_re != 0 else 0, 4)

    neg_re = re_real[re_real < 0]
    std_neg_re = neg_re.std() if not neg_re.empty else 0
    sortino = round(mean_re / std_neg_re * 100 if std_neg_re != 0 else 0, 4)

    ##############################################################################################

    data_1_new.to_csv('./temp/' + str(j) + 'x.csv', index=False)

    ###### 计算最大回撤
    cum_max_open = data_1_new['open'].cummax()  ###### 累计最大值
    drawdown = cum_max_open - data_1_new['open']  ###### 回撤值
    s = np.argmax(drawdown)  ###### 最大回撤结束位置

    ###### 确定最大回撤开始位置
    if s == 0:
        e = 0
    else:
        e = np.argmax(data_1_new['open'].iloc[:s]) 

    maxdrawdown = data_1_new['open'].iloc[e] - data_1_new['open'].iloc[s]  ###### 最大回撤
    drawdown_days = s - e  ###### 回撤持续周期数
    
    
    start_DAY = data_1_new.index[s] ######开始回撤的日期
    end_DAY = data_1_new.index[e] ######结束回撤的日期
    start_net_value = data_1_new[data_1_new.index == start_DAY]['open'].values[0] ######开始回撤的净值
    end_net_value = data_1_new[data_1_new.index == end_DAY]['open'].values[0] ######结束回撤的净值
    fig=plt.figure(figsize=(20,11))  
    plt.plot(data_1_new['eob'], data_1_new['open'])
    plt.plot([start_DAY, end_DAY], [start_net_value, end_net_value], linestyle='--', color='r')

    plt.xticks(range(0,data_1_size,int(data_1_size/m_size))) 
    
    plt.legend(['All:' + str(round(data_1_new['open'].iloc[-1]*100,2)) + '%' +
                '   ' + str(m_size) + 'm'
                '   Year:'+ str(round(data_1_new['open'].iloc[-1]/m_size*100*12,2)) + '%' +
                '   CalmarY:'+ str(round((data_1_new['open'].iloc[-1]/m_size*100*12)/(maxdrawdown*100),2)) +
                '   WP:' + str(round(wp_win_a/(wp_win_a + wp_lost_a)*100,2)) + '%' +
                '   RRR:' + str(round(rrr_win/(rrr_win+abs(rrr_lost))*100,2)) + '%' + ' / ' + str(round(rrr_win/abs(rrr_lost),2)) +
                '   T/N:' + str(wp_win_a + wp_lost_a ) + ' / ' + str(wp_nothing_a) +
                '   Sharpe:' + str(sharpe) +
                '   Sortino:' + str(sortino) +
                '   A0_MSE:' + str(round(cv_mse_A, 4)) +
                '   B0_MSE:' + str(round(cv_mse_B, 4)),
                'MD:'+ str(round(maxdrawdown*100,2)) + '%' +
                '   DA:'+ str(round(downarea,4)) + '%' +
                '   MDT:' + str(drawdown_days)+
                '   Date:' + str(data_1_new['eob'].iloc[e]) + ' - ' + str(data_1_new['eob'].iloc[s])] ,
                loc='upper left',fontsize = 11)
    plt.plot(data_1_new['eob'], data_1_new['down'], color='#ec700a')
    plt.fill_between(data_1_new['eob'], data_1_new['down'], 0, where=(data_1_new['down']<0), facecolor='#FF0000', alpha=0.1)
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig(f"./temp/{j}sy.jpg")
    plt.close()

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1_new['eob'], data_1_new['high'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig(f"./temp/{j}p.jpg")
    plt.close()
    
    ##############################################################################################
    ### 保存双概率的评估结果
    resP_A.append({'MSE_no': j, 'min_MSE_A': cv_mse_A})
    resP_B.append({'MSE_no': j, 'min_MSE_B': cv_mse_B})
    resR_A.append({'RMSE_no': j, 'min_RMSE_A': cv_rmse_A})
    resR_B.append({'RMSE_no': j, 'min_RMSE_B': cv_rmse_B})
    resF_A.append({'R2_no': j, 'max_R2_A': cv_r2_A})
    resF_B.append({'R2_no': j, 'max_R2_B': cv_r2_B})
    
    ##############################################################################################

    max_all = round(data_1_new['open'].iloc[-1]*100,2)
    res1.append({'All_no': j, 'max_All': max_all})

    max_CalmarY = round((data_1_new['open'].iloc[-1]/m_size*100*12)/(maxdrawdown*100),2)
    res2.append({'CalmarY_no': j, 'max_CalmarY': max_CalmarY})

    res3.append({'Downarea_no': j, 'min_Downarea': downarea})

    max_wp = round(wp_win_a/(wp_win_a + wp_lost_a)*100,2)
    res4.append({'WP_no': j, 'max_WP': max_wp})

    max_rrr = round(rrr_win/(rrr_win+abs(rrr_lost))*100,2)
    res5.append({'RRR_no': j, 'max_RRR': max_rrr})

    res6.append({'Sharpe_no': j, 'max_Sharpe': sharpe})
    res7.append({'Sortino_no': j, 'max_Sortino': sortino})



### A0的指标
aaaP_A = pd.DataFrame(resP_A)
aaaR_A = pd.DataFrame(resR_A)
aaaF_A = pd.DataFrame(resF_A)
bbbP_A = aaaP_A.sort_values(by="min_MSE_A",ascending=True).reset_index(drop=True)
bbbR_A = aaaR_A.sort_values(by="min_RMSE_A",ascending=True).reset_index(drop=True)
bbbF_A = aaaF_A.sort_values(by="max_R2_A",ascending=False).reset_index(drop=True)
bbbP_A['RMSE_no'] = bbbR_A['RMSE_no']
bbbP_A['min_RMSE_A'] = bbbR_A['min_RMSE_A']
bbbP_A['R2_no'] = bbbF_A['R2_no']
bbbP_A['max_R2_A'] = bbbF_A['max_R2_A']
bbbP_A.to_csv("./temp/Best_A.csv",index=False)

### B0的指标
aaaP_B = pd.DataFrame(resP_B)
aaaR_B = pd.DataFrame(resR_B)
aaaF_B = pd.DataFrame(resF_B)
bbbP_B = aaaP_B.sort_values(by="min_MSE_B",ascending=True).reset_index(drop=True)
bbbR_B = aaaR_B.sort_values(by="min_RMSE_B",ascending=True).reset_index(drop=True)
bbbF_B = aaaF_B.sort_values(by="max_R2_B",ascending=False).reset_index(drop=True)
bbbP_B['RMSE_no'] = bbbR_B['RMSE_no']
bbbP_B['min_RMSE_B'] = bbbR_B['min_RMSE_B']
bbbP_B['R2_no'] = bbbF_B['R2_no']
bbbP_B['max_R2_B'] = bbbF_B['max_R2_B']
bbbP_B.to_csv("./temp/Best_B.csv",index=False)


aaa1 = pd.DataFrame(res1)
aaa2 = pd.DataFrame(res2)
aaa3 = pd.DataFrame(res3)
aaa4 = pd.DataFrame(res4)
aaa5 = pd.DataFrame(res5)
aaa6 = pd.DataFrame(res6)
aaa7 = pd.DataFrame(res7)

bbb1 = aaa1.sort_values(by="max_All",ascending=False).reset_index(drop=True)
bbb2 = aaa2.sort_values(by="max_CalmarY",ascending=False).reset_index(drop=True)
bbb3 = aaa3.sort_values(by="min_Downarea",ascending=False).reset_index(drop=True)
bbb4 = aaa4.sort_values(by="max_WP",ascending=False).reset_index(drop=True)
bbb5 = aaa5.sort_values(by="max_RRR",ascending=False).reset_index(drop=True)
bbb6 = aaa6.sort_values(by="max_Sharpe",ascending=False).reset_index(drop=True)
bbb7 = aaa7.sort_values(by="max_Sortino",ascending=False).reset_index(drop=True)

bbb1['CalmarY_no'] = bbb2['CalmarY_no']
bbb1['max_CalmarY'] = bbb2['max_CalmarY']
bbb1['Downarea_no'] = bbb3['Downarea_no']
bbb1['min_Downarea'] = bbb3['min_Downarea']
bbb1['WP_no'] = bbb4['WP_no']
bbb1['max_WP'] = bbb4['max_WP']
bbb1['RRR_no'] = bbb5['RRR_no']
bbb1['max_RRR'] = bbb5['max_RRR']
bbb1['Sharpe_no'] = bbb6['Sharpe_no']
bbb1['max_Sharpe'] = bbb6['max_Sharpe']
bbb1['Sortino_no'] = bbb7['Sortino_no']
bbb1['max_Sortino'] = bbb7['max_Sortino']

bbb1.to_csv("./temp/Best_1.csv",index=False)